# Imports and installation


In [40]:
%%capture
!pip install datasets transformers accelerate evaluate wandb nltk pandas

In [41]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
import random
from dataclasses import dataclass
import numpy as np
from transformers import BartTokenizer, BartForConditionalGeneration, T5ForConditionalGeneration, AutoTokenizer
import pytorch_lightning as L

BATCH_SIZE = 128
SEED = 124

#DATA USED
SHORT = False
RANDOMIZED_SHORT = False
MEDIUM = True

MAX_SEQ_LEN = 512 if MEDIUM else 256

#MODEL USED
FEEDFORWARD = False
CONV1D = True
RNN = False
SEQ2SEQ = False

#MODEL CHOICES FOR SEQ2SEQ: bart-base, bart-large, t5-base
MODEL = "bart-base"

torch.manual_seed(SEED)
L.seed_everything(SEED)

124

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Data Preparation

In [43]:
if SHORT:
  df = pd.read_csv('/kaggle/input/full-dataset/shorthex2hex.csv')
elif RANDOMIZED_SHORT:
  df = pd.read_csv('/kaggle/input/full-dataset/randomized_shorthex2hex.csv')
elif MEDIUM:
  df = pd.read_csv('/kaggle/input/full-dataset/mediumhex2hex.csv')


df = df[:40960]

In [44]:
if RNN:
  df['text_hex'] = 'S' + df['text_hex'] + 'E'
  df['deflate_hex'] = 'S' + df['deflate_hex'] + 'E'

df.head()

,text,text_hex,deflate_hex
0,One of the other reviewers has mentioned that,4f6e65206f6620746865206f7468657220726576696577...,789cf3cf4b55c84f5328c9005240a248a128b52c33b53c...
1,A wonderful little production. <br /><br />The...,4120776f6e64657266756c206c6974746c652070726f64...,789c735428cfcf4b492d4a2bcd51c8c92c29c949552828...
2,I thought this was a wonderful way to,492074686f75676874207468697320776173206120776f...,789cf35428c9c82f4dcf2801d299c50ae589c50a890ae5...
3,Basically there's a family where a little boy,4261736963616c6c79207468657265277320612066616d...,789c734a2cce4c4eccc9a95428c9482d4a552f56485448...
4,"Petter Mattei's ""Love in the Time of Money""",506574746572204d6174746569277320224c6f76652069...,789c0b482d29492d52f04d045299eac50a4a3ef965a90a...


Instead of using the standard \<EOS> and \<SOS> tags we're using the letter S and E since they are not present in the vocabulary

In [45]:
if SEQ2SEQ:
    df = df[:15000]
    df[['deflate_hex', 'text_hex', 'text']] += "</s>"

ds = Dataset.from_pandas(df)
ds_train_test = ds.train_test_split(test_size=0.2, seed=SEED)
ds_test_dev = ds_train_test['test'].train_test_split(test_size=0.5, seed=SEED)
ds_splits = DatasetDict({
    'train': ds_train_test['train'],
    'valid': ds_test_dev['train'],
    'test': ds_test_dev['test']
})

## Data tokenization

In [46]:
token2id = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "a": 10, "b": 11, "c": 12, "d": 13, "e": 14, "f": 15, "P":16, "S": 17, "E":18 }

def create_id2token_vocab(token_to_id):
    id2token = {}
    for token, id in token_to_id.items():
        id2token[id] = token

    return id2token

id2token = create_id2token_vocab(token2id)


In [47]:
if SEQ2SEQ:
    if (MODEL == "bart-base"):
        tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
        model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

    elif (MODEL == "bart-large"):
        tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
        model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

    else:
        tokenizer = AutoTokenizer.from_pretrained("t5-base")
        model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [48]:
# Corrected collate_fn function
def collate_fn(batch):

    if FEEDFORWARD or CONV1D:

        texts = [elem['text_hex'] for elem in batch]
        encoded_hexs = [[token2id[char] for char in text] for text in texts]

        outputs = [elem['deflate_hex'] for elem in batch]
        encoded_outputs = [[token2id[char] for char in output] for output in outputs]

        # # Pad the sequences to MAX_SEQ_LEN chars with the padding token
        padded_hex = [torch.Tensor(encoded_hex + [token2id["P"]] * (MAX_SEQ_LEN - len(encoded_hex))) for encoded_hex in encoded_hexs]
        padded_outputs = [torch.Tensor(encoded_output + [token2id["P"]] * (MAX_SEQ_LEN - len(encoded_output))) for encoded_output in encoded_outputs]

        # # Stack the sequences
        padded_hex = torch.stack(padded_hex).long()
        padded_outputs = torch.stack(padded_outputs).long()


        return {
            'inputs': padded_hex,
            'outputs': padded_outputs
        }
    
    elif RNN:
        
        def pad_sequences(sequences, maxlen, value=token2id['P']):
            padded_sequences = []
            for sequence in sequences:
                padded_sequence = sequence[:maxlen]
                padded_sequence.extend([value] * (maxlen - len(padded_sequence)))

                padded_sequence = sequence +  [value] * (maxlen - len(sequence))
                padded_sequences.append(padded_sequence)

            return padded_sequences


        texts = [elem['text_hex'] for elem in batch]
        encoded_hex = [[token2id[x] for x in hex] for hex in texts]


        outputs = [elem['deflate_hex'] for elem in batch]
        encoded_outputs = [[token2id[x] for x in hex] for hex in outputs]


        maxlen = 0
        for seq in encoded_hex:
            if len(seq) > maxlen:
                maxlen = len(seq)
        for seq in encoded_outputs:
            if len(seq) > maxlen:
                maxlen = len(seq)

        padded_encoded_hex = pad_sequences(encoded_hex, maxlen)
        padded_encoded_outputs = pad_sequences(encoded_outputs, maxlen)


        return {
            'inputs': torch.tensor(padded_encoded_hex),
            "outputs": torch.tensor(padded_encoded_outputs)
        }
    
    elif SEQ2SEQ:
        inputs = [x["text_hex"] for x in batch]
        outputs = [x["deflate_hex"] for x in batch]
        input_features = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=MAX_SEQ_LEN)
        output_features = tokenizer(outputs, return_tensors="pt", padding=True, truncation=True, max_length=MAX_SEQ_LEN)["input_ids"]
        output_features[output_features == tokenizer.pad_token_id] = -100
        return {"input_ids": input_features["input_ids"], "attention_mask": input_features["attention_mask"], "labels": output_features}


In [49]:
train_dataloader = DataLoader(ds_splits['train'], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers = 3)
val_dataloader = DataLoader(ds_splits['valid'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers = 3)
test_dataloader = DataLoader(ds_splits['test'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers = 3)

torch.set_printoptions(profile="full")

In [50]:
import nltk
from nltk.metrics.distance import edit_distance

def decode_output(output):
    return ''.join([id2token[int(id)] for id in output])

def decode_input(input):
    return ''.join([id2token[int(id)] for id in input])

def compute_seq2seq_metrics(preds, labels, tokenizer):
    # Ensure labels with -100 are replaced by pad_token_id
    labels = torch.where(labels == -100, tokenizer.pad_token_id, labels)

    # Convert tensors to lists for decoding if they're not already in CPU
    if torch.is_tensor(preds):
        preds = preds.detach().cpu().tolist()
    if torch.is_tensor(labels):
        labels = labels.detach().cpu().tolist()

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    distances = [edit_distance(p, l) for p, l in zip(decoded_preds, decoded_labels)]
    avg_distance = np.mean(distances)
    count_unzippable = distances.count(0)

    return {"average_edit_distance": avg_distance, "count_unzippable": count_unzippable}

def evaluate(_device, _print, _training):

    model.eval()
    total_distance = 0
    total = 0

    distances_list = []


    for batch in test_dataloader:
            
        if FEEDFORWARD or CONV1D:
            x = batch["inputs"].to(_device)
            y = batch["outputs"].to(_device)

            y_hat = model(x, y)
            y_hat = torch.argmax(y_hat, dim=-1)

            output = decode_output(y[0])
            output_hat = decode_output(y_hat[0])

            output = [x for x in output if x != "P"]
            output_hat = [x for x in output_hat if x != "P"]
            
            distance = edit_distance(output, output_hat)

        elif RNN:
            x = batch["inputs"].transpose(0,1).to(_device)
            y = batch["outputs"].transpose(0,1).to(_device)

            y_hat = model(x, y)
            y_hat = torch.argmax(y_hat, dim=-1)
            
            y = y.transpose(0,1)
            y_hat = y_hat.transpose(0,1)
            
            assert len(y) == len(y_hat)
            
            for i in range(len(y)):
                output = decode_output(y[i])
                output_hat = decode_output(y_hat[i])

                output = [x for x in output if x != "P"]
                output_hat = [x for x in output_hat if x != "P"]

                first_eos_index = 0
                for i in range(len(output_hat)):
                    if output_hat[i] == "E":
                        first_eos_index = i
                        break

                # REMOVE START OF SEQUENCE TOKEN
                output = output[1:]
                output_hat = output_hat[1:first_eos_index]
                distance = edit_distance(output, output_hat)
                distances_list.append(distance)

        if _print:
            print(f"output = {output}")
            print(f"output_hat = {output_hat}")

        total_distance += distance
        total += 1

        if distance == 0:
            print(f"DISTANCE = 0!")
            print(f"output = {output}")
            print(f"output_hat = {output_hat}")

        if _training:
            return total_distance/total

    return (total_distance/total, distances_list)

In [51]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from pytorch_lightning.callbacks import EarlyStopping
from transformers import get_linear_schedule_with_warmup

INPUT_DIM = MAX_SEQ_LEN
EMBED_DIM = 128
HIDDEN_DIM = 512
OUTPUT_DIM = len(token2id)
LEARNING_RATE = 5e-4
DROPOUT_RATE = 0.5
NUM_HEADS = 4
NUM_LAYERS = 4
WEIGHT_DECAY = 0.01
MAX_EPOCHS = 10

#ATTENTION HYPERPARAMETERS (ONLY FOR FEEDFORWARD)
ATTENTION_USED = True

#RNN HYPERPARAMETERS
BIDIRECTIONAL = False
RNN_TYPE = 'RNN'  # Options: 'LSTM', 'GRU', 'RNN'

class FeedForward(pl.LightningModule):

    def __init__(self, input_dim=INPUT_DIM, embed_dim = EMBED_DIM, hidden_dim=HIDDEN_DIM, output_dim=OUTPUT_DIM, learning_rate=LEARNING_RATE,
                 dropout_rate=DROPOUT_RATE, bidirectional=BIDIRECTIONAL, num_layers=NUM_LAYERS, optimizer_type=AdamW, scheduler_type=StepLR,
                 scheduler_step_size=5, scheduler_gamma=0.1):
        super().__init__()
        self.save_hyperparameters()

        if FEEDFORWARD:
            self.embed = nn.Embedding(input_dim, embed_dim)
            self.positional_embeddings = nn.Parameter(torch.zeros(BATCH_SIZE, input_dim, embed_dim))
            nn.init.normal_(self.positional_embeddings, mean=0, std=embed_dim ** -0.5)  # Initialize positional embeddings
            self.self_attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=NUM_HEADS, dropout=dropout_rate, batch_first = True)
            self.fc1 = nn.Linear(embed_dim, hidden_dim)
            self.norm1 = nn.LayerNorm(hidden_dim)
            self.dropout = nn.Dropout(dropout_rate)
            self.fc2 = nn.Linear(hidden_dim, hidden_dim//2)
            self.norm2 = nn.LayerNorm(hidden_dim//2)
            self.fc3 = nn.Linear(hidden_dim//2, output_dim)

        elif CONV1D:

            # Embedding layer to transform dictionary indices into dense vectors
            self.embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=embed_dim)

            # Convolutional layers
            self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=embed_dim, kernel_size=3, padding=1)

            # Fully connected layers for classification
            self.fc1 = nn.Linear(embed_dim, embed_dim)  
            self.fc2 = nn.Linear(embed_dim, output_dim)

            # Hyperparameters
            self.learning_rate = learning_rate

        elif RNN:
            self.rnn_type = RNN_TYPE
            self.embedding = nn.Embedding(output_dim, embed_dim, padding_idx=token2id['P'])
            
            if self.rnn_type == 'LSTM':
                rnn_cell = nn.LSTM
            elif self.rnn_type == 'GRU':
                rnn_cell = nn.GRU
            else:  # Default to RNN if neither LSTM nor GRU is selected
                rnn_cell = nn.RNN
            
            self.encoder_rnn = rnn_cell(embed_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout_rate if num_layers > 1 else 0)
            self.decoder_rnn = rnn_cell(embed_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout_rate if num_layers > 1 else 0)
            
            self.dropout = nn.Dropout(dropout_rate)
            self.output_dim = output_dim
            self.linear = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
            self.criterion = nn.CrossEntropyLoss()
            
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, target, teacher_forcing_ratio=0.5):

        if FEEDFORWARD:

            # Embedding
            x = self.embed(x)  # Shape: [Batch, Seq_len, Embed_dim]

            if (ATTENTION_USED):   
                
                # Add positional embeddings
                positions = self.positional_embeddings
                x = x + positions
                
                # Self-attention
                attn_output, _ = self.self_attention(x, x, x)
            
                x = torch.relu(self.norm1(self.fc1(attn_output)))

            else:
                x = torch.relu(self.norm1(self.fc1(x)))
                
            x = self.dropout(x)
            x = torch.relu(self.norm2(self.fc2(x)))
            x = self.fc3(x)
            
            return x
        
        elif CONV1D:
            
            # Embedding layer
            x = self.embedding(x) 

            # Transpose from (batch_size, sequence_length, embedding_dim) to (batch_size, embedding_dim, sequence_length)
            x = x.permute(0, 2, 1)

            x = torch.relu(self.conv1(x))

            x = x.permute(0, 2, 1)

            x = torch.relu(self.fc1(x))

            x = self.fc2(x)
            
            return x
        
        elif RNN:
            target_len = target.shape[0]
            batch_size = target.shape[1]
            target_vocab_size = self.output_dim

            outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)

            x = self.dropout(self.embedding(x))
            rnn_output, h = self.encoder_rnn(x)

            x = target[0]
            for t in range(1, target_len):
                x = self.dropout(self.embedding(x.unsqueeze(0)))
                out, h = self.decoder_rnn(x, h if self.rnn_type in ['LSTM', 'GRU'] else None)
                predictions = self.linear(out)
                predictions = predictions.squeeze(0)
                outputs[t] = predictions
                pred = predictions.argmax(1)
                x = target[t] if random.random() < teacher_forcing_ratio else pred

            return outputs

    def configure_optimizers(self):
        optimizer = self.hparams.optimizer_type(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = self.hparams.scheduler_type(optimizer, step_size=self.hparams.scheduler_step_size, gamma=self.hparams.scheduler_gamma)
        return [optimizer], [scheduler]

    def step(self, batch):
        if (FEEDFORWARD or CONV1D):
            x = batch["inputs"]
            y = batch["outputs"]
            y = y.view(y.shape[0] * y.shape[1])
            y_hat = self(x, y)
            y_hat = y_hat.view(y_hat.shape[0] * y_hat.shape[1], y_hat.shape[2])

        elif RNN:
            inputs, targets = batch['inputs'], batch['outputs']
            inputs = inputs.transpose(0, 1)
            targets = targets.transpose(0, 1)

            output = self(inputs, targets)
            output_dim = output.shape[-1]

            y_hat = output.reshape(-1, output_dim)
            y = targets.reshape(-1)

        loss = self.loss(y_hat, y)
        return loss

    def training_step(self, batch, batch_idx):  
        loss = self.step(batch)
        self.log('train_loss', loss, prog_bar = True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('val_loss', loss, prog_bar = True)
        return loss
    

class Seq2Seq(pl.LightningModule):
    def __init__(self, tokenizer, model):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model

    def forward(self, input_ids, attention_mask=None, labels=None):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self.forward(**batch)
        self.log('train_loss', outputs.loss, prog_bar=True, logger=True)
        return outputs.loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(**batch)
        self.log('val_loss', outputs.loss, prog_bar=True, logger=True)

        preds = torch.argmax(outputs.logits, dim=-1)
        metrics = compute_seq2seq_metrics(preds, batch['labels'], self.tokenizer)
        for key, value in metrics.items():
            self.log(f'{key}', value, prog_bar=True, logger=True)

        return outputs.loss

    def test_step(self, batch, batch_idx):
        outputs = self.forward(**batch)
        self.log('test_loss', outputs.loss, prog_bar=True, logger=True)
        
        preds = torch.argmax(outputs.logits, dim=-1)
        metrics = compute_seq2seq_metrics(preds, batch['labels'], self.tokenizer)
        
        for key, value in metrics.items():
            self.log(f'{key}', value, prog_bar=True, logger=True)
        return outputs.loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = {
            'scheduler': get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=10000),
            'name': 'learning_rate',
            'interval': 'step',
            'frequency': 1
        }
        return [optimizer], [scheduler]


if SEQ2SEQ:
    model = Seq2Seq(tokenizer, model)
    trainer = pl.Trainer(
        precision='16-mixed',
        max_epochs=MAX_EPOCHS,
        enable_progress_bar=True,
        callbacks=[EarlyStopping(monitor='val_loss', patience=3)]
    )
    trainer.fit(model, train_dataloader, val_dataloader)
    trainer.test(model, test_dataloader)
else:
    model = FeedForward()
    trainer = pl.Trainer(max_epochs=MAX_EPOCHS)
    trainer.fit(model, train_dataloader, val_dataloader)
    print(evaluate(_device = "cpu", _print = True, _training= False))

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

output = ['7', '8', '9', 'c', 'f', '3', '5', '4', '2', 'f', '4', 'b', '5', '5', '2', '8', '4', 'e', '4', 'd', 'c', 'd', '5', '3', '2', '8', 'c', '9', 'c', '8', '2', 'c', '5', '6', '4', '8', 'c', 'b', 'c', 'c', 'c', '9', '0', '5', 'b', '2', '8', 'a', '5', '2', '5', '3', '1', '5', '4', 'a', '3', '2', '7', '3', '5', '3', '8', 'b', '1', '5', '1', '2', 'f', '3', '5', '2', '1', '4', '5', '2', '1', '3', '9', '3', '3', '3', '0', '0', '1', 'f', '0', '8', '0', 'e', '3', '5']
output_hat = ['7', '8', '9', 'c', 'c', '8', '4', '8', 'c', 'c', '4', '8', '5', '8', '2', '8', 'c', '9', '4', 'c', '4', '8', '5', 'c', '2', '8', 'c', '9', 'c', '8', '2', 'c', 'c', 'd', '4', '8', 'c', 'b', 'c', 'c', '4', '8', '5', 'c', '2', '8', '5', '8', '2', '8', 'c', 'c', '4', '8', '4', 'd', '2', 'c', 'c', 'c', 'c', '9', 'c', '8', '2', 'c', '5', 'c', '4', '8', '4', '8', '2', '8', '2', 'c', '5', '8', '2', 'f', '0']
output = ['7', '8', '9', 'c', '0', 'b', 'c', '9', 'c', '8', '2', 'c', '5', '6', '2', '8', 'c', 'e', 'c', '8', '

# Model

# Training